# Introduction
We recently had a meeting where we discussed which metabolites should be added to the biomass reaction as we know they are critical for certain steps in metabolism. In this notebook, I will try to add them to the biomass reaction and check their biosynthesis is complete. To get an indication of the stoichiometry to use, I will try to find experimental data for measurements of the presence of these compounds.

List of metabolites to add to biomass reaction:
- Coenzyme B12
- Biotin
- sperimine and spermidine
- Heme

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite, Reaction

In [3]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [5]:
model_e_coli = cameo.load_model ('iML1515')

In [5]:
model_b_sub = cameo.load_model('iYO844')

## Biotin
We know that Biotin is a critical cofactor for our strain. Therefore here I will check how to add biotin to the strain and then make sure the biosynthetic pathway is functional as it should be. 

E. coli has biotin present at a stoichiomerty of -2e-06 mmol/gCDW. It is missing in the bacillus biomass equation. As I cannot find any experimental data about the amount in bacillus or geobacillus, we will use the amount of the E. coli model as an estimate. It is also so small that it will not constitute a huge change of fluxed so this should be alright.

In [5]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [6]:
model.reactions.biomass.add_metabolites({
    model.metabolites.btn_c:-2e-6
})

Again, adding biotin kills biomass accumulation, so I need to check where this problem comes from and fix it. Right now, biotin is only associated to biotin5-amp. not to its biosynthesis, so I have to add this probably. It also seems a lot of the metabolites are missing. The biotin biosynthesis pathway is annotated in our strain, so should be added.

Note: I will work from biotin back up.

BTNS: this reaction incorporates a sulfur into the dethiobiotin. It is a bit unclear where exactly the sulfur comes from. the B. subtilis model circumvents this by just adding an elemental sulfur (s_c) into the reaction, but this is a dead end reaction so not a feasible solution. In E. coli it has been shown and in literature further hypothesized that the sulphur comes from one Fe2S2 cluster that is sacrificed. As we have no indication of how this may go otherwise, this will be applied here. 
the Fe2S1 cluster must then also be recycled. 

In [7]:
#first step converting dethiobiotin to biotin, using SAM
#add dethiobiotin met first
model.add_metabolites(Metabolite(id='dtbt_c', name = 'Dethiobiotin', compartment= 'c', formula = 'C10H17N2O3', charge = -1))

In [8]:
model.metabolites.dtbt_c.annotation = model_b_sub.metabolites.dtbt_c.annotation

In [9]:
#add 2fe2s_c metabolite
model.add_metabolites(Metabolite(id= '2fe2s_c', name = '[2Fe-2S] iron-sulfur cluster', charge = 2 , compartment = 'c', formula = 'S2Fe2' ))

In [10]:
model.metabolites.get_by_id('2fe2s_c').annotation = model_e_coli.metabolites.get_by_id('2fe2s_c').annotation

In [11]:
#add 2fe1s_c metabolite
model.add_metabolites(Metabolite(id= '2fe1s_c', name = '[2Fe-2S] desulfurated iron-sulfur cluster', charge = 2 , compartment = 'c', formula = 'SFe2' ))

In [12]:
model.metabolites.get_by_id('2fe1s_c').annotation = model_e_coli.metabolites.get_by_id('2fe1s_c').annotation

In [13]:
#add reaction 
model.add_reaction(Reaction(id='BTS5', name = 'biotin synthase'))

In [14]:
model.reactions.BTS5.annotation['kegg.reaction']='R01078'
model.reactions.BTS5.annotation['ec-code']='2.8.1.6'
model.reactions.BTS5.annotation['sbo']='SBO:0000176'

In [15]:
model.groups.get_by_id('00780 - Biotin metabolism').add_members(model.reactions.BTS5)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:93: UserWarning:

need to pass in a list



In [16]:
model.reactions.BTS5.add_metabolites({
    model.metabolites.btn_c:1,
    model.metabolites.met__L_c:1,
    model.metabolites.dad_5_c:1,
    model.metabolites.dtbt_c:-1,
    model.metabolites.amet_c:-1,
    model.metabolites.get_by_id('2fe2s_c'):-1,
    model.metabolites.get_by_id('2fe1s_c'):1,
    model.metabolites.h_c:1
})

In [17]:
model.reactions.BTS5.gene_reaction_rule = 'BioB'

In [18]:
model.reactions.BTS5.check_mass_balance()

{}

We also need to be sure the 2fe2s_c can be regenerated. In E. coli this is done via the I2FE2SR reaction, which is the general sulfur donor. In our model, we have this encoded as [Enzyme]-S-sulfanylcysteine (enzscys) which can be regenerated through reaction with a cysteine. So here, I will include regenerate of the 2fe2s metabolite via a reaction with the enzscys metabolite. (the [Enzyme]-S-sulfanylcysteine metabolite is also the sulfur donor in thiamine biosynthesis in our strain and model)

In [19]:
model.add_reaction(Reaction(id='I2FE2SR', name = '[2Fe-2S] regeneration'))

In [20]:
model.reactions.I2FE2SR.annotation['metanetx.reaction'] = 'MNXR100766'
model.reactions.I2FE2SR.annotation['sbo']='SBO:0000176'

In [21]:
model.groups.get_by_id('00780 - Biotin metabolism').add_members(model.reactions.I2FE2SR)

In [22]:
model.reactions.I2FE2SR.add_metabolites({
    model.metabolites.get_by_id('2fe1s_c'):-1,
    model.metabolites.get_by_id('2fe2s_c'):1,
    model.metabolites.enzscys_c:-1,
    model.metabolites.enzcys_c:1
})

Now that dethiobiotin can be made, we need to work through the rest of the upstream part.

In [23]:
#add 7,8-Diaminononanoate met
model.add_metabolites(Metabolite(id='dann_c', name = '7,8-Diaminononanoate', compartment = 'c', formula = 'C9H21N2O2', charge = 1))

In [24]:
model.metabolites.dann_c.annotation = model_e_coli.metabolites.dann_c.annotation

In [25]:
#add reaction DBTS
model.add_reaction(Reaction(id='DBTS', name = 'Dethiobiotin synthase'))

In [26]:
model.reactions.DBTS.annotation = model_e_coli.reactions.DBTS.annotation

In [27]:
model.groups.get_by_id('00780 - Biotin metabolism').add_members(model.reactions.DBTS)

In [28]:
model.reactions.DBTS.gene_reaction_rule = 'BioD'

In [29]:
model.reactions.DBTS.add_metabolites({
    model.metabolites.atp_c:-1,
    model.metabolites.dann_c:-1,
    model.metabolites.co2_c:-1,
    model.metabolites.adp_c:1,
    model.metabolites.pi_c:1,
    model.metabolites.dtbt_c:1,
    model.metabolites.h_c:3
})

In [30]:
#add next step from 8-amino-7-oxopentanoate
#add metabolite
model.add_metabolites(Metabolite(id='8aonn_c', name = '8-Amino-7-oxononanoate', compartment = 'c', charge = 0, formula = 'C9H17NO3'))

In [31]:
model.metabolites.get_by_id('8aonn_c').annotation = model_e_coli.metabolites.get_by_id('8aonn_c').annotation

In [32]:
#add amob_c metabolite, a byproduct in the reaction
model.add_metabolites(Metabolite(id='amob_c', name = 'S-Adenosyl-4-methylthio-2-oxobutanoate', compartment = 'c', formula = 'C15H19N5O6S', charge = 0))

In [33]:
model.metabolites.amob_c.annotation = model_e_coli.metabolites.amob_c.annotation

In [34]:
#add reactions AMAOTr
model.add_reaction(Reaction(id='AMAOTr', name = 'Adenosylmethionine-8-amino-7-oxononanoate transaminase'))

In [35]:
model.reactions.AMAOTr.annotation = model_e_coli.reactions.AMAOTr.annotation

In [36]:
model.groups.get_by_id('00780 - Biotin metabolism').add_members(model.reactions.AMAOTr)

In [37]:
model.reactions.AMAOTr.gene_reaction_rule = 'BioD'

In [38]:
model.reactions.AMAOTr.add_metabolites({
    model.metabolites.get_by_id('8aonn_c'):-1,
    model.metabolites.amet_c:-1,
    model.metabolites.dann_c:1,
    model.metabolites.amob_c:1
})

In [39]:
#here I need to add a sink for the little bit on amob_c that is formed, as there is no other indication as to what happens with this metabolite.
model.add_boundary(model.metabolites.amob_c, type = 'sink', reaction_id = 'DM_amob_c', lb = 0, ub = 1000, sbo_term = 'SBO:0000628')

Reaction identifier,DM_amob_c
Name,S-Adenosyl-4-methylthio-2-oxobutanoate sink
Memory address,0x014ff445f308
Stoichiometry,amob_c --> S-Adenosyl-4-methylthio-2-oxobutanoate -->
GPR,
Lower bound,0
Upper bound,1000


The 8-Amino-7-oxononanoate is made from pimeloyl-ACP or pimeloyl-Coa. I need to check where this pimeloyl-ACP/CoA met is and where it comes from. Manandhar et al. (2017,  doi:10.1111/mmi.13648 ) have shown that this comes from pimelate, via the BioW enzyme in B. subtilis. 
The same article argues that the pimelate-Coa is produced via pimeloyl-ACP. Then BioI is a cytochrome P450 which can convert a long chain acyl-carrier protein into the desired pimeloyl-ACP. Blasting the B. sub BioI against our strains genome gives a significant hit. As we have no other indication how it may go in our strain, this is the best bet we have.

So I will add the conversion from pimeloyl-Coa to the 8aonn_c metabolite. And create pimeloyl synthesis through the pimeloyl-ACP and this from a long-chain fatty acid.

In [40]:
#Add pimeloyl-Coa met
model.add_metabolites(Metabolite(id='pimcoa_c', name = 'Pimeloyl-CoA', compartment = 'c', formula = 'C28H46N7O19P3S', charge = 0))

In [41]:
model.metabolites.pimcoa_c.annotation['kegg.compound'] = 'C01063'
model.metabolites.pimcoa_c.annotation['chebi'] = 'CHEBI:15504'
model.metabolites.pimcoa_c.annotation['sbo'] = 'SBO:0000247'

In [42]:
#add AOXSr reaction
model.add_reaction(Reaction(id='AOXSr', name= '8-amino-7-oxononanoate synthase'))

In [43]:
model.reactions.AOXSr.annotation['sbo']= 'SBO:0000176'
model.reactions.AOXSr.annotation['kegg.reaction']='R03210'
model.reactions.AOXSr.annotation['ec-code']= '2.3.1.47'

In [44]:
model.groups.get_by_id('00780 - Biotin metabolism').add_members(model.reactions.AOXSr)

In [45]:
model.reactions.AOXSr.add_metabolites({
    model.metabolites.ala__L_c:-1,
    model.metabolites.pimcoa_c:-1,
    model.metabolites.get_by_id('8aonn_c'):1,
    model.metabolites.coa_c:1,
    model.metabolites.co2_c:1,
    model.metabolites.h_c:4
})

In [46]:
#add pimeloyl-ACP met
model.add_metabolites(Metabolite(id='pimACP_c', name = 'Pimeloyl-[acyl-carrier protein]', compartment = 'c', formula = 'C18H31N2O10PRS', charge = -1))

In [47]:
model.metabolites.pimACP_c.annotation = model_e_coli.metabolites.pimACP_c.annotation

In [48]:
#add Coa to ACP change
model.add_reaction(Reaction(id='PMEACP', name = 'Pimelate-Coa:ACP transferase'))

In [49]:
model.reactions.PMEACP.annotation['sbo'] = 'SBO:0000176'

In [50]:
model.groups.get_by_id('00780 - Biotin metabolism').add_members(model.reactions.PMEACP)

In [51]:
model.reactions.PMEACP.add_metabolites({
    model.metabolites.pimcoa_c:-1,
    model.metabolites.ACP_c:-1,
    model.metabolites.pimACP_c:1,
    model.metabolites.coa_c:1,
    model.metabolites.h_c:4
})

In [52]:
model.reactions.PMEACP.bounds = (-1000,1000)

Stok and De Voss (2000, doi:10.1006/abbi.2000.2067) have shown that the B. subtilis BioI has predominant preference for C16-fatty acid chains. So we will use this to assume the pimeloyl-ACP is made from the cleavage of the C16-ACP metabolite. This would result in the formation of nonanal: an aldehyde aóften used in essential oils. One reason why the resulting aldehydes that have been formed from this reaction are poorly studied is because they are hard to measure. They are often volatile or are oxidized to the corresponding acid. FOr that reason, as we would have no clear indication what happens with the Nonanal, i will add a sink reaction for it. Again, this is only very small amounts of carbon we would loose.But this is better than adding incorrect pathways here.

In [53]:
#add nonanal metabolite
model.add_metabolites(Metabolite(id='nonal_c', name = 'Nonanal', compartment = 'c', charge = 0, formula = 'C9H18O'))

In [54]:
model.metabolites.nonal_c.annotation = model_b_sub.metabolites.nonal_c.annotation

In [55]:
#add reaction
model.add_reaction(Reaction(id='PMES', name = 'pimeloyl-[acyl-carrier protein] synthase'))

In [56]:
model.reactions.PMES.annotation['sbo']= 'SBO:0000176'
model.reactions.PMES.annotation['ec-code']= '1.14.14.46'
model.reactions.PMES.annotation['kegg.reaction']= 'R10123'

In [57]:
model.groups.get_by_id('00780 - Biotin metabolism').add_members(model.reactions.PMES)

In [58]:
model.reactions.PMES.add_metabolites({
    model.metabolites.palmACP_c:-1,
    model.metabolites.fmnh2_c:-2,
    model.metabolites.o2_c:-3,
    model.metabolites.pimACP_c:1,
    model.metabolites.nonal_c:1,
    model.metabolites.fmn_c:2,
    model.metabolites.h2o_c:3
})

Finally, need to add the nonanal sink.

In [59]:
model.add_boundary(model.metabolites.nonal_c, type = 'sink', reaction_id = 'DM_nonal_c', lb = 0, ub = 1000, sbo_term = 'SBO:0000628')

Reaction identifier,DM_nonal_c
Name,Nonanal sink
Memory address,0x014ff44ed188
Stoichiometry,nonal_c --> Nonanal -->
GPR,
Lower bound,0
Upper bound,1000


Now the ability to grow has been restored, due to the complete Biotin pathway. Though, note two demands had to be added here.

In [60]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

## Vitamin B12
'Coenzyme B12 is involved in methylmalonylCoA mutase and methionine synthase. I am not sure how critical the former is but methionine synthase is essential on a minimal medium. So a small amount of B12 is needed.' Argument from David.

So considering that, vitamin B12 should be added into the biomass reaction. E. coli has the adobcbl present with a stoichiometry of -0.000223 mmol/gCDW. This could be a good baseline to use as estimate.

In [5]:
#add vitB12 to biomass, see if it still grows to check biosynthesis pathway
model.reactions.biomass.add_metabolites({
    model.metabolites.adocbl_c:-0.000223
})

Adding the vitB12 kills biomass accumulation, so now we have to figure out where in the pathway this breaks. 

In [6]:
with model:
    model.add_boundary(model.metabolites.rdmbzi_c, type = 'sink', lb = -1000, ub = 1000, reaction_id = 'test')
    print (model.optimize().objective_value)
    

0.6996423935419391


Already we see that the step from agdpcbi_c into adocbl_c causes an issue. Supplying rdmbzi_c causes the problem, so likely something prevents this metabolite from being made that blocks this whole pathway. supplying dmbzid_c also alows growth, so it must be somewhere in the synthesis of this that the issue arises. This metabolite can be made from FMNH2. It seems this reaction may be missing. So I need to check the presence of EC 1.13.11.79 in our strain.

The presence of this enzyme has been confirmed only in Propionibacterium freudenreichii. running a blast with this enzyme showed that there is a significant hit in the genome of our organism. This together with the experimental evidence that the strain can grow aerobically without the addition of VitB12 highloights that this enzyme should likely be present. So I will add it here.

This reaction creates a side product: dialurate. I've tried to find out what happens with this metabolite in metabolism, but cannot find any indication in literature what would happen. Therefore, I will have to add a demand reaction here. This gap can later be filled when more knowledge is acquired.

Note: this enzyme is oxygen dependent and so can be an issue when growing anaerobically. 

In [7]:
#add dialurate metabolite
model.add_metabolites(Metabolite(id='diur_c', name = 'dialurate', compartment = 'c', charge = -1, formula = 'C4H3N2O4'))

In [8]:
model.metabolites.diur_c.annotation['sbo'] = 'SBO:0000247'
model.metabolites.diur_c.annotation['chebi'] = 'CHEBI:140629'

In [9]:
#add reaction
model.add_reaction(Reaction(id = 'BLUB', name = '5,6-dimethylbenzimidazole synthase'))

In [10]:
model.reactions.BLUB.annotation['sbo'] = 'SBO:0000176'
model.reactions.BLUB.annotation['ec-code'] = '1.13.11.79'
model.reactions.BLUB.annotation['kegg.reaction'] = 'R09083'

In [11]:
model.reactions.BLUB.add_metabolites({
    model.metabolites.fmnh2_c:-1,
    model.metabolites.o2_c:-1,
    model.metabolites.dmbzid_c:1,
    model.metabolites.e4p_c:1,
    model.metabolites.diur_c:1,
    model.metabolites.h_c:1
})

In [12]:
#add demand for dialurate
model.add_boundary(model.metabolites.diur_c, type = 'sink', reaction_id = 'DM_diur_c', lb = 0, ub = 1000, sbo_term = 'SBO:0000628')

Reaction identifier,DM_diur_c
Name,dialurate sink
Memory address,0x015b8f07e7c8
Stoichiometry,diur_c --> dialurate -->
GPR,
Lower bound,0
Upper bound,1000


In [13]:
#check demand carries very little flux
model.metabolites.diur_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,BLUB,100.0,0.000156,fmnh2_c + o2_c --> diur_c + dmbzid_c + e4p_c +...
CONSUMING,DM_diur_c,100.0,0.000156,diur_c -->


With fixing this issue, we now have restored growth when the VitB12 requirement is in the biomass reaction. Doing this should also have unblocked the whole vitamin B12 biosythesis pathway, which should decrease the % of blocked reactions by a few % to 14%. See memote report '../reports/2020-05-18-c67aa07.html.


In [14]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

Here we add the vitamin B12 requirement to the biomass reaction. However, I've also noticed that we should revist the complete VitB12 metabolism, as this is currently going through the aerobic pathway. Beata mentions something about this in her thesis. So I will make sure to look into this in another notebook.

## Spermine (sprm_c) and spermidine (spmd_c)
Geobacilli ensure RNA and DNA stability in high temperatures through use of protamines, spermidines, spermines, tRNA methyltransferase and spermidine synthase. Therefore we need to add the spermidine and spermine into the biomass equation, as these are two crucial factors involved in the thermophili of the genus. 

__Stoichiometry__
A paper by Maheswar et al. (2004, doi:10.1111/j.1365-2672.2004.02395.x ) haslooked into the distribution of putrescince, spermine and spermidine in Geobacillus thermoleovorans over cultivation time and also at different temperatures. Here we see that The aboslute levels of the compounds really change at different tempertures and between growth vs. sporulation.
From a physiological point of view, it makes sense to estimate the stoichiometries in our model based on data of growing strains at 60°C. From the paper, we can then estimate the stoichiometeries to be:
- spermine: 2.8 umol/gCDW = -0.0028 mmol/gCDW
- spermidine: 1.2 umol/gCDW = -0.0012 mmol/gCDW
- putrescine: 6.0 umol/gCDW = -0.006 mmol/gCDW

Previously, we estimated putrescine levels from the e. coli model. This lead to a coefficient of -0.03327 mmol/gCDW. This is quite a bit higher than what the data shows, so this should be changed.
In E. coli the spermidine is also present in the biomass reaction at a stoichiometry of -0.006744 mmol/gCDW. This is in the same range as the data we have found, so I will use the ratios found in the paper. 

In [6]:
model.reactions.biomass.add_metabolites({
    model.metabolites.sprm_c: -0.0028,
    model.metabolites.spmd_c: -0.0012,
    model.metabolites.ptrc_c: 0.02727
})

In [7]:
model.optimize().objective_value

0.7005524280322071

The model still grows fine, so this would indicate that the biosyntheis pathways here are all complete. Then we can save & commit and move on to the next element to add.

In [8]:
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

## Heme
 porphyrins are critical co-factors in many reactions, leading us to believe that this should be a biomass requirement. Specifically, heme should be included in the biomass demand. The E. coli model has three different hemes (hemeO, protoheme, and siroheme) present in the biomass reaction, each at the same stoichiometry: -0.000223. Our strain has all the genes annotated that are necessary for the production of these compounds, so it may make sense to add the three different heme types into our biomass equation.
 
hemeO and pheme can be added to the biomass reaction without any issue. But siroheme is currently not present in the model. So i will add this metabolite and check the pathway to it here.

Note: good, extensive review on heme biosynthesis in prokaryotes:Dailey et al., 2017 (. https://doi.org/10.1128/MMBR.00048-16 )

In [2]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [3]:
#add sheme metabolite
model.add_metabolites(Metabolite(id= 'sheme_c', name = 'Siroheme', compartment = 'c', formula = 'C42H36FeN4O16', charge = -8))

In [6]:
model.metabolites.sheme_c.annotation = model_e_coli.metabolites.sheme_c.annotation

In [7]:
model.reactions.biomass.add_metabolites({
    model.metabolites.hemeO_c: -0.000223,
    model.metabolites.pheme_c: -0.000223,
    model.metabolites.sheme_c: -0.000223
})

sheme should connect through Sirohydrochlorin, a metabolite also used in the anaerobic pathway of vitamin B12 synthesis.

In [8]:
#add sirohydrochlorin met
model.add_metabolites(Metabolite(id='scl_c', name = 'Sirohydrochlorin', compartment = 'c', formula ='C42H39N4O16', charge = -7))  

In [9]:
model.metabolites.scl_c.annotation = model_e_coli.metabolites.scl_c.annotation

In [10]:
#add reaction from scl to sheme
model.add_reaction(Reaction(id='SHCHF', name = 'Sirohydrochlorin ferrochelatase'))

In [11]:
model.reactions.SHCHF.annotation = model_e_coli.reactions.SHCHF.annotation

In [12]:
model.groups.get_by_id('00860 - Porphyrin and chlorophyll metabolism').add_members(model.reactions.SHCHF)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:93: UserWarning:

need to pass in a list



In [13]:
model.reactions.SHCHF.add_metabolites({
    model.metabolites.fe2_c :-1,
    model.metabolites.scl_c:-1,
    model.metabolites.h_c:3,
    model.metabolites.sheme_c:1
})

In [14]:
#add reaction to scl from precorrin2
model.add_reaction(Reaction(id='SHCHD2', name = 'Sirohydrochlorin dehydrogenase (NAD)'))

In [15]:
model.reactions.SHCHD2.annotation = model_e_coli.reactions.SHCHD2.annotation

In [16]:
model.groups.get_by_id('00860 - Porphyrin and chlorophyll metabolism').add_members(model.reactions.SHCHD2)

In [17]:
model.reactions.SHCHD2.add_metabolites({
    model.metabolites.shcl_c:-1,
    model.metabolites.nad_c:-1,
    model.metabolites.scl_c:1,
    model.metabolites.nadh_c:1
})

In [18]:
model.optimize().objective_value

0.7002547478325231

Now we've fixed the biosynthesis and our model can grow again. 

In [19]:
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')